In [1]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [2]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tdam.cover import CubicalCover
from tdam.clustering import FailSafeClustering
from tdam.core_old import MapperAlgorithm

from TrafficTDApythonUtils.utils_v3 import *
from TrafficTDApythonUtils.plots import *

In [ ]:
import pandas as pd

new_grid = pd.read_csv("../ComputedData/ForTDA/new_grid.csv")
new_grid.drop(columns=['hotspot'], inplace=True)

In [ ]:
countyname = new_grid['COUNTYNAME']
new_grid.drop(columns=['COUNTYNAME'], inplace=True)
lens = PCA(10).fit_transform(new_grid.to_numpy())

# lens_df = pd.DataFrame(lens, columns=[f'PC{i+1}' for i in range(lens.shape[1])])

In [ ]:
overlap = 7
interval = 10
detailed_results = []
silhouette_for_intervals = []

mapper_algo = MapperAlgorithm(
    cover=CubicalCover(
        n_intervals=interval,
        overlap_frac=overlap / 10
    ),
    clustering=FailSafeClustering(
        KMeans(n_clusters=2)
    ),
    n_jobs=-1
)

mapper_info = mapper_algo.fit_transform(new_grid.to_numpy(), lens)

silhouette_for_intervals.append(mapper_info[1])
result = {
    "overlap": overlap,
    "interval": interval,
    "silhouette": mapper_info[1],
    "mapper_info": mapper_info
}
detailed_results.append(result)
detailed_results_df = pd.DataFrame(detailed_results)

In [ ]:
choose = '車輛撞擊部位大類別名稱-最初_汽車'

mapper_plotter = MapperPlotter(detailed_results_df['mapper_info'][0], 
                               new_grid, seed=87, iterations=30, dim=2,
                                range_lst=[-0.5, 0.5, 0.5, -0.5])

In [ ]:
def avg_label(data):
    return sum(data) / len(data) if len(data) > 0 else 0

mapper_plot = mapper_plotter.create_mapper_plot(choose, avg_label, avg=True)
full_info = mapper_plotter.extract_data()
mapper_plotter.map_colors(choose, size=0, threshold=0)
mapper_plotter.plot(choose, avg=True, set_label=True, size=1000, anchor=1.33)